In [15]:
import mne
import math
import numpy as np
import neurokit2 as nk
import pandas as pd
import pickle
import random
import os

from scipy.io import loadmat
from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from os.path import exists


In [16]:
participants = 89
excluded_numbers = [8, 35, 39, 81]
label_file_path='./details/labels.csv'
subjects = [idx for idx in range(1, participants+1) if idx not in excluded_numbers]
random.shuffle(subjects)
print(len(subjects))
subjects_train = subjects[:int(len(subjects)*0.8)]
subjects_test = subjects[int(len(subjects)*0.8):]
print(len(subjects_train), len(subjects_test))

window_time = 2

85
68 17


In [17]:
def load_data(subject):
    if exists('./../../eeg_data/{}.mat'.format(subject)):
        file = './../../eeg_data/{}.mat'.format(subject)
        mat_data = loadmat(file)
        eeg_data = (mat_data['EEG']['data'][0, 0])*1e-6
        channel_names = [ch[0] for ch in mat_data['EEG']['chanlocs'][0, 0]['labels'][0]]
        sampling_freq = mat_data['EEG']['srate'][0, 0][0, 0]
        return eeg_data, channel_names, sampling_freq

In [18]:
def get_features(select_data, label):

    shape = np.shape(select_data)
    # 计算要拆分的段数
    num_segments = shape[1] // (window_time*250)
    select_data = select_data[:, 0:num_segments*window_time*250]
    # 使用 `np.split()` 函数进行拆分
    segments = np.split(select_data, num_segments, axis=1)

    features = []
    for seg in segments:
        seg_fft = []
        for i in range(seg.shape[0]):
            # [:64]表示只取前64个频率分量的结果
            fft = np.abs(np.fft.fft(seg[i, :]))[0:64]
            fft = fft / np.sum(fft)
            seg_fft.append(fft)
            # print("fft.type: ", type(fft))
        feat = np.concatenate(seg_fft, axis = 0)
        # print(len(feat))   # 128
        features.append((feat, label))
    return features

In [12]:
train_data = []
test_data = []

for i in subjects_train:
    df = pd.read_csv(label_file_path)

    # 读取这个csv文件的Tension (1, 8, 15, 21, 28, and 35)列中ID为i的行的值，如果这个值小于等于15，那么label为0，否则为1
    tension_value = df.loc[df['ID'] == i, 'Tension (1, 8, 15, 21, 28, and 35)'].values[0]
    if tension_value<=15:
        label = 0
    else:
        label = 1

    eeg_data, channel_names, sampling_freq = load_data(i)
    # print(eeg_data.shape, channel_names, sampling_freq)

    eeg_data_fp1 = eeg_data[channel_names.index('FP1'), : ]
    eeg_data_fp2 = eeg_data[channel_names.index('FP2'), : ]
    # print(eeg_data_fp1.shape, eeg_data_fp2.shape)

    # 把eeg_data_fp1和eeg_data_fp2的数据拼接在一起，成为一个二维数组
    eeg_data_fp1 = eeg_data_fp1.reshape(1, -1)
    
    eeg_data_fp2 = eeg_data_fp2.reshape(1, -1)
    eeg_data = np.concatenate((eeg_data_fp1, eeg_data_fp2), axis=0)

    features = get_features(eeg_data, label)

    train_data += features

for i in subjects_test:
    df = pd.read_csv(label_file_path)

    # 读取这个csv文件的Tension (1, 8, 15, 21, 28, and 35)列中ID为i的行的值，如果这个值小于等于15，那么label为0，否则为1
    tension_value = df.loc[df['ID'] == i, 'Tension (1, 8, 15, 21, 28, and 35)'].values[0]
    if tension_value<=15:
        label = 0
    else:
        label = 1

    eeg_data, channel_names, sampling_freq = load_data(i)
    # print(eeg_data.shape, channel_names, sampling_freq)

    eeg_data_fp1 = eeg_data[channel_names.index('FP1'), : ]
    eeg_data_fp2 = eeg_data[channel_names.index('FP2'), : ]
    # print(eeg_data_fp1.shape, eeg_data_fp2.shape)

    # 把eeg_data_fp1和eeg_data_fp2的数据拼接在一起，成为一个二维数组
    eeg_data_fp1 = eeg_data_fp1.reshape(1, -1)

    eeg_data_fp2 = eeg_data_fp2.reshape(1, -1)
    eeg_data = np.concatenate((eeg_data_fp1, eeg_data_fp2), axis=0)

    features = get_features(eeg_data, label)

    test_data += features


In [13]:
# 指定要保存的文件名和文件夹路径,如果没有的话生成这个文件夹
folder_name = 'samples'
if not os.path.exists(folder_name):
    os.mkdir(folder_name)
filename = os.path.join(folder_name, 'window_{}s_train.pkl'.format(window_time))

# 使用 pickle.dump() 将数据保存到文件中
with open(filename, 'wb') as file:
    pickle.dump(train_data, file)

filename = os.path.join(folder_name, 'window_{}s_test.pkl'.format(window_time))

# 使用 pickle.dump() 将数据保存到文件中
with open(filename, 'wb') as file:
    pickle.dump(test_data, file)

print("数据已保存到文件:", filename)

数据已保存到文件: samples\window_2s_test.pkl
